<a href="https://colab.research.google.com/github/SleepingSkipper/PDF_CLEANER/blob/main/T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [1]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2
!pip install -q transformers==4.4.2 pytorch_lightning==1.2.1 sentencepiece

In [2]:
!pip install bert_score

In [3]:
from os import path
import pandas as pd
import math
import tarfile
import re
ROOT = "/content/drive/MyDrive/Colab_Data/Tanshin"


In [4]:
# 事前学習済みモデル
PRETRAINED_MODEL_NAME = "sonoisa/t5-base-japanese"

# 転移学習済みモデル
MODEL_DIR = "./drive/MyDrive/Colab_Data/Tanshin/increased_datasets3"
LOG_DIR = "./drive/MyDrive/Colab_Data/Tanshin/increased_datasets3_logs"
CHECKPOINT_DIR="./drive/MyDrive/Colab_Data/Tanshin/increased_datasets_checkpoints"

In [5]:
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [6]:
import tarfile
import re

def remove_brackets(text):
    text = re.sub(r"(^【[^】]*】)|(【[^】]*】$)", "", text)
    return text

def normalize_text(text):
    assert "\n" not in text and "\r" not in text
    text = text.replace("\t", " ")
    text = text.strip()
    text = normalize_neologd(text)
    text = text.lower()
    return text

def read_title_body(file):
    next(file)
    next(file)
    title = next(file).decode("utf-8").strip()
    title = normalize_text(remove_brackets(title))
    body = normalize_text(" ".join([line.decode("utf-8").strip() for line in file.readlines()]))
    return title, body

In [8]:
df =pd.read_csv("./drive/MyDrive/Colab_Data/Tanshin/data_for_T5_increased_improved.csv")

In [9]:
all_data=[]
for i, rows in df.iterrows():
    text = rows["important_texts"]
    abstract_text = rows["summary"]

    text = text.replace("\n","")
    abstract_text = abstract_text.replace("\n","")
    text = normalize_text(text)
    abstract_text = normalize_text(abstract_text)
    if len(text) > 0:
        all_data.append({
            # "body": "重要文:"+text,
            "body": "summarize: "+text,
            "title": abstract_text
            })

In [10]:
print(all_data)

[{'body': 'summarize: ヤマハ。当連結会計年度の売上収益は、新型コロナウイルスの感染拡大に伴う影響等により、為替のマイナス影響23億円を含め、前期に対し415億97百万円(10.0%)減少の3,726億30百万円となりました。事業利益は、為替のマイナス影響6億円を含め、前期に対し56億41百万円(12.2%)減少の407億11百万円となりました。親会社の所有者に帰属する当期利益は、事業利益の減少に加え、新型コロナウイルスの感染拡大に伴う操業停止損23億18百万円、減損損失35億53百万円の計上もあり、前期に対し80億6百万円(23.1%)減少の266億15百万円となりました。(楽器事業)アコースティックピアノと電子楽器は、中国での販売が成長軌道に復帰した他、各国の市況が回復基調にあるものの、商品供給不足もあり上期の減収をカバーするまでには至らず、減収となりました。管楽器は、市況の回復が遅れ減収となりました。ギターは、国内や中国で販売を伸ばし増収となりました。以上により、当事業の売上収益は、前期に対し為替のマイナス影響17億円を含め、303億90百万円(11.3%)減少の2,389億81百万円となりました。事業利益は、為替のマイナス影響8億円を含め、53億32百万円(14.1%)減少の324億17百万円となりました。(音響機器事業)オーディオ機器は、ステイホーム需要によりサウンドバー等の販売が伸長しましたが、上期の減収をカバーするまでには至らず、全体では減収となりました。業務用音響機器は、ライブ市場や設備市場の停滞により減収となりました。以上により、当事業の売上収益は、前期に対し為替のマイナス影響4億円を含め、105億79百円(9.2%)減少の1,038億13百万円となりました。事業利益は、為替のプラス影響2億円を含め、15億4百万円(17.5%)減少の70億67百万円となりました。(その他の事業)電子デバイスは減収となりましたが、自動車用内装部品やfa機器は需要の回復により増収となりました。以上により、当事業の売上収益は、前期に対し6億26百万円(2.1%)減少の298億36百万円となりました。事業利益は、前期30百万円に対し、11億95百万円増加の12億25百万円となりました。②次期の見通し2022年3月期連結業績については、新型コロナウイルス

## データ分割

データセットを90% : 5%: 5% の比率でtrain/dev/testに分割します。

* trainデータ: 学習に利用するデータ
* devデータ: 学習中の精度評価等に利用するデータ
* testデータ: 学習結果のモデルの精度評価に利用するデータ

In [11]:
import random
from tqdm import tqdm

random.seed(1234)
random.shuffle(all_data)

def to_line(data):
    title = data["title"]
    body = data["body"]

    assert len(title) > 0 and len(body) > 0
    return f"{title}\t{body}\n"

data_size = len(all_data)
train_ratio, dev_ratio, test_ratio = 0.9, 0.05, 0.05

with open(path.join(ROOT, "data","increased_datasets2","train.tsv"), "w", encoding="utf-8") as f_train, \
    open(path.join(ROOT, "data","increased_datasets2","dev.tsv"), "w", encoding="utf-8") as f_dev, \
    open(path.join(ROOT, "data","increased_datasets2","test.tsv"), "w", encoding="utf-8") as f_test:
# with open(path.join(ROOT,"train.tsv"), "w", encoding="utf-8") as f_train, \
#     open(path.join(ROOT,"dev.tsv"), "w", encoding="utf-8") as f_dev, \
#     open(path.join(ROOT, "test.tsv"), "w", encoding="utf-8") as f_test:
    
    for i, data in tqdm(enumerate(all_data)):
        line = to_line(data)
        if i < train_ratio * data_size:
            f_train.write(line)
        elif i < (train_ratio + dev_ratio) * data_size:
            f_dev.write(line)
        else:
            f_test.write(line)

71it [00:00, 8837.71it/s]


# 学習に必要なクラス等の定義

学習にはPyTorch/PyTorch-lightning/Transformersを利用します。

In [12]:
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

# 乱数シードの設定
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [13]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

import matplotlib.pyplot as plt
from matplotlib import rc
# from bert_score import score

In [14]:
# GPU利用有無
USE_GPU = torch.cuda.is_available()

# 各種ハイパーパラメータ
args_dict = dict(
    data_dir=path.join(ROOT, "data","increased_datasets2"),
    # data_dir=path.join(ROOT),  # データセットのディレクトリ
    model_name_or_path=PRETRAINED_MODEL_NAME,
    tokenizer_name_or_path=PRETRAINED_MODEL_NAME,

    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    gradient_accumulation_steps=1,

    # max_input_length=512,
    # max_target_length=64,
    # train_batch_size=8,
    # eval_batch_size=8,
    # num_train_epochs=4,

    n_gpu=1 if USE_GPU else 0,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)


## TSVデータセットクラス

TSV形式のファイルをデータセットとして読み込みます。  
形式は"{title}\t{body}\t{genre_id}"です。

In [15]:
class TsvDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, input_max_len=512, target_max_len=512):
        self.file_path = os.path.join(data_dir, type_path)
        
        self.input_max_len = input_max_len
        self.target_max_len = target_max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()
  
    def __len__(self):
        return len(self.inputs)
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        source_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": source_mask, 
                "target_ids": target_ids, "target_mask": target_mask}

    def _make_record(self, title, body):
        # ニュースタイトル生成タスク用の入出力形式に変換する。
        input = f"{body}"
        target = f"{title}"
        return input, target
  
    def _build(self):
        with open(self.file_path, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip().split("\t")
                assert len(line) == 2
                assert len(line[0]) > 0
                assert len(line[1]) > 0

                title = line[0]
                body = line[1]

                input, target = self._make_record(title, body)

                tokenized_inputs = self.tokenizer.batch_encode_plus(
                    [input], max_length=self.input_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                tokenized_targets = self.tokenizer.batch_encode_plus(
                    [target], max_length=self.target_max_len, truncation=True, 
                    padding="max_length", return_tensors="pt"
                )

                self.inputs.append(tokenized_inputs)
                self.targets.append(tokenized_targets)


試しにテストデータ（test.tsv）を読み込み、トークナイズ結果をみてみます。

In [16]:
# トークナイザー（SentencePiece）モデルの読み込み
tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL_NAME, is_fast=True)

# テストデータセットの読み込み
train_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "train.tsv", 
                           input_max_len=1024, target_max_len=128)

In [17]:
for data in train_dataset:
    print("A. 入力データの元になる文字列")
    print(tokenizer.decode(data["source_ids"]))
    print()
    print("B. 入力データ（Aの文字列がトークナイズされたトークンID列）")
    print(data["source_ids"])
    print()
    print("C. 出力データの元になる文字列")
    print(tokenizer.decode(data["target_ids"]))
    print()
    print("D. 出力データ（Cの文字列がトークナイズされたトークンID列）")
    print(data["target_ids"])
    break

A. 入力データの元になる文字列
summarize: 住友金属鉱山。当期の平均為替レートは前期と比べ円高となりました。この結果、当期の銅及びニッケル並びに金価格はいずれも前期を上回りました。このような状況のなか、当期の連結売上高は、電池材料事業での一時的な生産調整による減販などはあったものの、銅及びニッケル並びに金価格が前期を上回ったことなどにより、前期に比べ741億76百万円増加し、9,261億22百万円となりました。また、前期にも本会計方針の変更を遡及適用し、売上高を表示しております。連結税引前当期利益は、増収及び持分法による投資損益が好転したことなどにより、前期に比べ443億44百万円増加し、1,233億79百万円となりました。親会社の所有者に帰属する当期利益は、連結税引前当期利益が増加したことなどにより、前期に比べ340億4百万円増加し、946億4百万円となりました。 2報告セグメントの状況(a)資源セグメント(単位:百万円)セグメント利益は、新型コロナウイルス感染症拡大の影響による販売量の減少及び、ケブラダ・ブランカ銅鉱山(チリ)における建設中断期間の費用計上などによる悪化があったものの、金及び銅価格の上昇などにより前期を上回りました。(うち非支配持分を除く当社持分は31.5%)(b)製錬セグメント(単位:百万円)(当社の主な製品別生産量)(注)生産量には、受委託分を含めて表示しております。(c)材料セグメント(単位:百万円)セグメント利益は、電池材料等では一時的な生産調整により販売量が減少しましたが、粉体材料等において前期と比較し販売量が増加したことなどにより、前期を上回りました。連結財政状態計算書(単位:百万円)当期末の資産合計は前期末に比べて増加しました。非鉄価格については、経済環境の好転期待や余剰となった資金がコモディティ市場にも流入していることから高値で推移しておりますが、需給環境からは乖離した水準と見られ急落するリスクも想定されます。このような環境の中で非鉄金属の需給は、銅・ニッケルともにほぼ均衡または若干の供給余剰と見込まれており、次期の価格については銅7,800$/t(当期6,879$/t)、ニッケル7.00$/lb(当期6.80$/lb)と予想しています。次期の連結業績予想につきましては、主要な非鉄金属価格は足元の水準を考慮の上で将来の

## 学習処理クラス

[PyTorch-Lightning](https://github.com/PyTorchLightning/pytorch-lightning)を使って学習します。

PyTorch-Lightningとは、機械学習の典型的な処理を簡潔に書くことができるフレームワークです。

In [18]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
#         self.hparams.update(hparams)
        # self.hparams.save_hyperparameters
        self.save_hyperparameters()
        
        # 事前学習済みモデルの読み込み
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)

        # トークナイザーの読み込み
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path, is_fast=True)

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, 
                decoder_attention_mask=None, labels=None):
        """順伝搬"""
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

    def _step(self, batch):
        """ロス計算"""
        labels = batch["target_ids"]

        # All labels set to -100 are ignored (masked), 
        # the loss is only computed for labels in [0, ..., config.vocab_size]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100
        
        # self を呼ぶ ≒ forwardメソッドを呼ぶ　≒ modelを呼ぶ　
        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            decoder_attention_mask=batch['target_mask'],
            labels=labels
        )
        # T5ForConditionalGeneration クラスの forward methodは、特殊メソッド __call__() をオーバーライドしているため、
        # インスタンス名をそのまま関数のように使った場合の処理が記述されている。
        # よって、outputsは、T5ForConditionalGenerationクラスのforwardが　returnする Seq2SeqLMOutputオブジェクト。
        # T5では、logitsとlabelsのクロスエントロピーロスを使っている。
        loss = outputs[0]
        logits = outputs[1]
        return loss, logits

    def training_step(self, batch, batch_idx):
        """訓練ステップ処理"""
        # loss = self._step(batch)
        # self.log("train_loss", loss)
        # print(loss)
        # return {"loss": loss}
        target = batch["target_ids"]
        loss, logits = self._step(batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": logits, "targets":target}

    def validation_step(self, batch, batch_idx):
        """バリデーションステップ処理"""
        # loss = self._step(batch)
        # self.log("val_loss", loss)
        # print(loss)
        # return {"val_loss": loss}
        target = batch["target_ids"]
        loss, logits = self._step(batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": logits, "targets":target}

    def test_step(self, batch, batch_idx):
        """テストステップ処理"""
        # loss = self._step(batch)
        # self.log("test_loss", loss)
        # return {"test_loss": loss}
        loss, logits = self._step(batch)
        self.log("test_loss", loss, prog_bar=True, logger=True)
        return loss





    def configure_optimizers(self):
        """オプティマイザーとスケジューラーを作成する"""
        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() 
                            if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() 
                            if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, 
                          lr=self.hparams.learning_rate, 
                          eps=self.hparams.adam_epsilon)
        self.optimizer = optimizer

        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=self.hparams.warmup_steps, 
            num_training_steps=self.t_total
        )
        self.scheduler = scheduler

        return [optimizer], [{"scheduler": scheduler, "interval": "step", "frequency": 1}]

    def get_dataset(self, tokenizer, type_path, args):
        """データセットを作成する"""
        return TsvDataset(
            tokenizer=tokenizer, 
            data_dir=args.data_dir, 
            type_path=type_path, 
            input_max_len=args.max_input_length,
            target_max_len=args.max_target_length)
    
    def setup(self, stage=None):
        """初期設定（データセットの読み込み）"""
        if stage == 'fit' or stage is None:
            train_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                             type_path="train.tsv", args=self.hparams)
            self.train_dataset = train_dataset

            val_dataset = self.get_dataset(tokenizer=self.tokenizer, 
                                           type_path="dev.tsv", args=self.hparams)
            self.val_dataset = val_dataset

            self.t_total = (
                (len(train_dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
            )

    def train_dataloader(self):
        """訓練データローダーを作成する"""
        return DataLoader(self.train_dataset, 
                          batch_size=self.hparams.train_batch_size, 
                          drop_last=True, shuffle=True, num_workers=4)

    def val_dataloader(self):
        """バリデーションデータローダーを作成する"""
        return DataLoader(self.val_dataset, 
                          batch_size=self.hparams.eval_batch_size, 
                          num_workers=4)

# 転移学習を実行

GPUのOut Of Memoryエラーが発生することがあります。
その場合、次の順にハイパーパラメータの調整を試してみるとエラーを解消できる場合があります。

1. 訓練時のバッチサイズ train_batch_size を小さくする（例えば4）。  
小さくしすぎると精度が悪化することがあります。
2. 入力文の最大トークン数 max_input_length や出力文の最大トークン数 max_target_length を小さくする（例えば、入力を256や出力を32にする）。  
入力文の最大トークン数を小さくすると一般に精度が落ち、出力文の最大トークン数を小さくすると生成できる文章の長さが短くなります。

In [19]:
logger = TensorBoardLogger(LOG_DIR , name="1125_increased_data_T5")
# 5エポック連続で損失が改善されなかった場合に、早期停止可能
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3)

In [20]:
checkpoint_callback = ModelCheckpoint(
  # 保存パスとファイル名を指定
  dirpath=CHECKPOINT_DIR,
  filename="1125_best-checkpoint_for_T5_increased_data",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory ./drive/MyDrive/Colab_Data/Tanshin/increased_datasets_checkpoints exists and is not empty.
  warnings.warn(*args, **kwargs)


In [22]:
# 学習に用いるハイパーパラメータを設定する
args_dict.update({
    "max_input_length":  1024,  # 入力文の最大トークン数
    "max_target_length": 128,  # 出力文の最大トークン数
    "train_batch_size":  2,  # 訓練時のバッチサイズ
    "eval_batch_size":   8,  # テスト時のバッチサイズ
    "num_train_epochs":  1,  # 訓練するエポック数
    })
args = argparse.Namespace(**args_dict)

# train_params = dict(
#     accumulate_grad_batches=args.gradient_accumulation_steps,
#     gpus=args.n_gpu,
# #     gpus=1,
#     max_epochs=args.num_train_epochs,
#     precision= 16 if args.fp_16 else 32,
#     amp_level=args.opt_level,
#     gradient_clip_val=args.max_grad_norm,
# )

train_params = dict(
    logger = logger,
    callbacks=[early_stopping_callback, checkpoint_callback],
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
#     gpus=1,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
)

In [23]:
# 転移学習の実行（GPUを利用すれば1エポック10分程度）
model = T5FineTuner(args)
trainer = pl.Trainer(**train_params)
trainer.fit(model)

# 最終エポックのモデルを保存
# model.tokenizer.save_pretrained(MODEL_DIR)
# model.model.save_pretrained(MODEL_DIR)

del model

GPU available: True, used: True
TPU available: None, using: 0 TPU cores

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Epoch 0, global step 31: val_loss reached 1.99118 (best 1.99118), saving model to "/content/drive/MyDrive/Colab_Data/Tanshin/increased_datasets_checkpoints/1125_best-checkpoint_for_T5_increased_data-v1.ckpt" as top 1
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [24]:
# test_step を実行
result= trainer.test()
print(result)

# ベストなモデルが保存されたパスを表示
print(trainer.checkpoint_callback.best_model_path)
# ベストなモデルの、validation dataに対する損失を表示
print(trainer.checkpoint_callback.best_model_score)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: you defined a test_step but have no test_dataloader. Skipping test loop
  warnings.warn(*args, **kwargs)


1
/content/drive/MyDrive/Colab_Data/Tanshin/increased_datasets_checkpoints/1125_best-checkpoint_for_T5_increased_data-v1.ckpt
tensor(1.9912, device='cuda:0')


# 学習済みモデルの読み込み

In [25]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer


model = T5FineTuner.load_from_checkpoint("./drive/MyDrive/Colab_Data/Tanshin/increased_datasets_checkpoints/1125_best-checkpoint_for_T5_increased_data-v1.ckpt")

# save the model in HF format with
model.model.save_pretrained(MODEL_DIR)
model.tokenizer.save_pretrained(MODEL_DIR)

# after this you can load the hf_model using
trained_model =T5ForConditionalGeneration.from_pretrained(MODEL_DIR)
tokenizer = T5Tokenizer.from_pretrained(MODEL_DIR, is_fast=True)

In [26]:
# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
    trained_model.cuda()

# 全テストデータの本文に対するタイトル生成

In [27]:
import textwrap
from tqdm.auto import tqdm
from sklearn import metrics

# テストデータの読み込み
test_dataset = TsvDataset(tokenizer, args_dict["data_dir"], "dev.tsv", 
                          input_max_len=args.max_input_length, 
                          target_max_len=args.max_target_length)

test_loader = DataLoader(test_dataset, batch_size=4, num_workers=4)

trained_model.eval()

inputs = []
outputs = []
targets = []

for batch in tqdm(test_loader):
    input_ids = batch['source_ids']
    input_mask = batch['source_mask']
    if USE_GPU:
        input_ids = input_ids.cuda()
        input_mask = input_mask.cuda()

    output = trained_model.generate(input_ids=input_ids, 
        attention_mask=input_mask, 
        max_length=args.max_target_length,
        temperature=1.0,          # 生成にランダム性を入れる温度パラメータ
        repetition_penalty=2.5,   # 同じ文の繰り返し（モード崩壊）へのペナルティ
        )

    output_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                            clean_up_tokenization_spaces=False) 
                for ids in output]
    target_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in batch["target_ids"]]
    input_text = [tokenizer.decode(ids, skip_special_tokens=True, 
                               clean_up_tokenization_spaces=False) 
                for ids in input_ids]

    inputs.extend(input_text)
    outputs.extend(output_text)
    targets.extend(target_text)
    

  0%|          | 0/1 [00:00<?, ?it/s]

## 生成結果確認

形式
- generated: 生成されたタイトル
- actual: 人が作成したタイトル（正解）
- body: ニュース記事の本文


In [28]:
for output, target, input in zip(outputs, targets, inputs):
    print("generated: " + output)
    print("actual:    " + target)
    print("body:      " + input)
    print()

generated: 、2023年3月期の営業利益は、新型コロナウイルス感染症の影響により、デバイス事業が減収となりました。2020年3月期の営業利益は、新型コロナウイルス感染症の影響により、医療部材を中心に販売費及び一般管理費が増加したものの、売上総利益は増収となりました。当社グループは、新型コロナウイルス感染症の影響により、米国・欧州地域や米国・欧州地域において新型コロナウイルス感染症の影響により、心臓血管系領域のptcaガイドワイヤーや貫通カテーテルなどの医療機器関連取引が減少したものの、2018年4-6月期通期連結業績予想を据え
actual:    朝日インテック、2021年6月期の営業利益実績は市場予想並みの水準で着地。2021年6月期の営業利益実績は市場予想並みの水準でした。2021年6月期の売上高は、新型コロナの影響により、デバイス事業が減収し、またメディカル事業においても、全地域において症例数が減少する影響を受けましたが、メディカル事業の海外市場において、自社ブランド製品の販売促進活動強化や直接販売の推進などにより増収となりました。同期の営業利益は、販売費及び一般管理費が増加したものの、前期比2.8%増と増益を確保しました
body:      重要文:朝日インテック。当社グループは、2023年までの5か年の中期経営計画「『asahi road to 1000』〜only one技術で強固なグローバルニッチno.1を目指す〜」に基づき、連結売上高1,000億円達成に向けた事業ポートフォリオの構築として、「グローバル市場の戦略的な開拓と患部・治療領域の拡大」を推進することで、これまでの基本戦略の集大成を図り、既存事業の収益基盤を強化、また将来に向けた成長への投資を継続することにより「グローバルニッチ市場における新規事業の創出」を実現し、グローバルニッチ市場における当社のプレゼンスの強化と企業価値の一層の向上を目指し、その成長戦略を支えるためのビジネス基盤として、「グローバル展開に最適な研究開発・生産体制の構築」を進めるとともに「持続的成長に向けた経営基盤の確立」を図ってまいりました。その背景といたしましては、新型コロナウイルス感染症の影響が想定外であったことや、上記3などのm&a戦略の実行は現中期経営計画の期限である2023年を超えて更に長期的な事業